In [21]:
import pandas as pd
import numpy as np
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
RAW_DATA_DIR = Path('C:\\Users\\loicc\\OneDrive - Efrei\\Bureau\\COURS\\M2\\S9\\Machine Learning in Production\\Data Pipeline\\mlops-nba')

# Remplacez 'nom_du_fichier.csv' par le nom de votre fichier CSV
filename = RAW_DATA_DIR / '2023-2024 NBA Player Stats - Regular.csv'
print(f"Running on file: {filename}")
players_df = pd.read_csv(filename, sep=',', encoding='Windows-1252')
players_csv = pd.read_csv(filename, sep=',', encoding='Windows-1252')

Running on file: C:\Users\loicc\OneDrive - Efrei\Bureau\COURS\M2\S9\Machine Learning in Production\Data Pipeline\mlops-nba\2023-2024 NBA Player Stats - Regular.csv


In [23]:
players_csv.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [24]:
from nba_api.stats.endpoints import playercareerstats

# Instantiate the playercareerstats class with a player_id
career = playercareerstats.PlayerCareerStats(player_id='2544')

# Get the data frames
career = career.get_data_frames()[0]

In [25]:
career.columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [26]:
import pandas as pd

# Colonnes de players_csv
players_columns = ['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
                   '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
                   'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

# Colonnes de career
career_columns = ['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
                  'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
                  'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
                  'BLK', 'TOV', 'PF', 'PTS']

# Trouver les colonnes manquantes dans career.columns
colonnes_manquantes = set(players_columns) - set(career_columns)

# Afficher les colonnes manquantes
print(colonnes_manquantes)


{'TRB', '3P%', 'Pos', 'FT', 'ORB', '3PA', '2P%', 'FG', 'G', 'Player', 'Rk', 'Tm', 'Age', 'MP', 'DRB', 'FT%', '2PA', '2P', '3P', 'FG%', 'eFG%'}


In [46]:
# Exemple d'utilisation de leagueleaders
from nba_api.stats.endpoints import leagueleaders
import pandas as pd

# Créer une instance de LeagueLeaders
league_leaders = leagueleaders.LeagueLeaders(season='2023-24')

# Convertir en DataFrame
basic_stats_df = league_leaders.get_data_frames()[0]

# Exemple d'utilisation de commonplayerinfo
from nba_api.stats.endpoints import commonplayerinfo

# Vous devrez probablement boucler sur les identifiants des joueurs pour obtenir leurs informations
player_info = commonplayerinfo.CommonPlayerInfo(player_id='2544')
additional_info_df = player_info.get_data_frames()[0]
additional_info_df['PLAYER_ID'] = additional_info_df['PERSON_ID']
# Fusionner les DataFrames sur une clé commune, comme l'ID du joueur
combined_df = pd.merge(basic_stats_df, additional_info_df, on='PLAYER_ID')

combined_df['eFG%'] = (combined_df['FGM'] + 0.5 * combined_df['FG3M']) / combined_df['FGA']
combined_df

,PLAYER_ID,RANK,PLAYER,TEAM_ID_x,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG,eFG%
0,2544,15,LeBron James,1610612747,LAL,40,1371,370,712,0.52,...,2003,2023,N,Y,Y,2003,1,1,Y,0.580056


In [40]:
from nba_api.stats.endpoints import leagueleaders, commonallplayers
import pandas as pd

# Créer une instance de LeagueLeaders
league_leaders = leagueleaders.LeagueLeaders(season='2023-24')
basic_stats_df = league_leaders.get_data_frames()[0]

# Utiliser CommonAllPlayers pour obtenir une liste de tous les joueurs
all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)
all_players_df = all_players.get_data_frames()[0]

# Fusionner les DataFrames sur une clé commune, comme l'ID du joueur
combined_df = pd.merge(basic_stats_df, all_players_df, left_on='PLAYER_ID', right_on='PERSON_ID', how='left')

# Calculer eFG%
combined_df['eFG%'] = (combined_df['FGM'] + 0.5 * combined_df['FG3M']) / combined_df['FGA']

# Afficher le DataFrame combiné
combined_df.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID_x,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,PLAYER_SLUG,TEAM_ID_y,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_SLUG,TEAM_CODE,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH,eFG%
0,1628983,1,Shai Gilgeous-Alexander,1610612760,OKC,42,1450,462,847,0.545,...,shai_gilgeous-alexander,1610612760,Oklahoma City,Thunder,OKC,thunder,thunder,Y,00,0.572609
1,203507,2,Giannis Antetokounmpo,1610612749,MIL,41,1444,473,784,0.603,...,giannis_antetokounmpo,1610612749,Milwaukee,Bucks,MIL,bucks,bucks,Y,00,0.614158
2,1629029,3,Luka Doncic,1610612742,DAL,36,1327,412,854,0.482,...,luka_doncic,1610612742,Dallas,Mavericks,DAL,mavericks,mavericks,Y,00,0.563817
3,203954,4,Joel Embiid,1610612755,PHI,32,1096,386,716,0.539,...,joel_embiid,1610612755,Philadelphia,76ers,PHI,sixers,sixers,Y,00,0.564944
4,203999,5,Nikola Jokic,1610612743,DEN,44,1480,450,763,0.590,...,nikola_jokic,1610612743,Denver,Nuggets,DEN,nuggets,nuggets,Y,00,0.619921


In [43]:
combined_df.columns

Index(['PLAYER_ID', 'RANK', 'PLAYER', 'TEAM_ID_x', 'TEAM', 'GP', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'EFF',
       'AST_TOV', 'STL_TOV', 'PERSON_ID', 'DISPLAY_LAST_COMMA_FIRST',
       'DISPLAY_FIRST_LAST', 'ROSTERSTATUS', 'FROM_YEAR', 'TO_YEAR',
       'PLAYERCODE', 'PLAYER_SLUG', 'TEAM_ID_y', 'TEAM_CITY', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_SLUG', 'TEAM_CODE', 'GAMES_PLAYED_FLAG',
       'OTHERLEAGUE_EXPERIENCE_CH', 'eFG%'],
      dtype='object')

In [29]:
combined_df.head(1)

,PLAYER_ID,RANK,PLAYER,TEAM_ID_x,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG,eFG%
0,2544,15,LeBron James,1610612747,LAL,40,1371,370,712,0.52,...,2003,2023,N,Y,Y,2003,1,1,Y,0.580056


In [47]:
from nba_api.stats.endpoints import leagueleaders, commonplayerinfo
import pandas as pd

# Créer une instance de LeagueLeaders
league_leaders = leagueleaders.LeagueLeaders(season='2023-24')

# Convertir en DataFrame
basic_stats_df = league_leaders.get_data_frames()[0]

# Liste pour stocker les DataFrames supplémentaires
additional_info_dfs = []

# Boucler sur les identifiants des joueurs
for player_id in basic_stats_df['PLAYER_ID']:
    # Obtenir des informations supplémentaires pour chaque joueur
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    additional_info_df = player_info.get_data_frames()[0]
    
    # Ajouter le DataFrame à la liste
    additional_info_dfs.append(additional_info_df)

# Concaténer tous les DataFrames supplémentaires
all_additional_info_df = pd.concat(additional_info_dfs)

# Fusionner les DataFrames sur une clé commune, comme l'ID du joueur
combined_df = pd.merge(basic_stats_df, all_additional_info_df, on='PLAYER_ID')

# Calculer eFG%
combined_df['eFG%'] = (combined_df['FGM'] + 0.5 * combined_df['FG3M']) / combined_df['FGA']

# Afficher le DataFrame combiné
print(combined_df)


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)